In [0]:
#setup
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count,countDistinct, round, when, sum as _sum, rand

In [0]:
# Iniciar Spark
spark = SparkSession.builder \
    .appName("Case ifood teste AB") \
    .getOrCreate()

In [0]:
#local dos arquivos
base_path = "/Volumes/workspace/default/case_ifood/"

In [0]:
#ler com spark
df_pedido = spark.read.json(base_path + "order.json.gz", multiLine=True)
df_restaurante = spark.read.option("header", "true").option("inferSchema", "true").csv(base_path + "restaurant.csv.gz")
df_usuario = spark.read.option("header", "true").option("inferSchema", "true").csv(base_path + "consumer.csv.gz")
df_teste = spark.read.option("header", True).option("inferSchema", True).csv(base_path + "ab_test_ref.csv")
df_estado = spark.read.option("header", True).option("inferSchema", True).option("sep", ";").csv(base_path + "ddd_estados.csv")

In [0]:
df_usuario.limit(1).show()
df_restaurante.limit(1).show()
df_teste.limit(1).show()
df_pedido.limit(1).show()
df_estado.limit(1).show()

+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+
|         customer_id|language|          created_at|active|customer_name|customer_phone_area|customer_phone_number|
+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+
|e8cc60860e09c0bb1...|   pt-br|2018-04-05 14:49:...|  true|         NUNO|                 46|            816135924|
+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+

+--------------------+--------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|                  id|          created_at|enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|merchant_zip_code| merchant_city|merchant_state|merchant_country|
+--------------------+---------------

**Entendimento df_usuario**

In [0]:
#entendimento df
print((df_usuario.count(), len(df_usuario.columns)))
display(df_usuario.limit(10))
print(df_usuario.columns)
df_usuario.printSchema()
df_usuario.describe().show()

(806156, 7)


customer_id,language,created_at,active,customer_name,customer_phone_area,customer_phone_number
e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e397ce65cba92f70928,pt-br,2018-04-05T14:49:18.165Z,true,NUNO,46,816135924
a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d58c40ab11eef19b6eb,pt-br,2018-01-14T21:40:02.141Z,true,ADRIELLY,59,231330577
41e1051728eba13341136d67d0757f8d8cc44b2a405b718a8c5efba2c093b2c0,pt-br,2018-01-07T03:47:15.554Z,true,PAULA,62,347597883
8e7c1dcb64edf95c935147f6d560cb068c44714cb1b21b287f1f433ae7b0b04e,pt-br,2018-01-10T22:17:08.160Z,true,HELTON,13,719366842
7823d4cf4150c5daeae0bd799206852fc123bdef0cf5d1639c8cb568d73e8c83,pt-br,2018-04-06T00:16:20.935Z,true,WENDER,76,543232158
77e6a36569f6c8f357a661ef59b7854c1701e538ceb5a3a7821dd44797944e0c,pt-br,2018-01-27T20:00:11.176Z,true,MICHELLE,90,806008944
0ce03c2b567f34f421372f0a8b70c6bfd04308d134d52b4436682b304f6d7dd8,pt-br,2018-01-04T14:12:16.843Z,true,MARLOS,18,724199206
7c2c4d1fe3ce743583c15365efb8ee4ab90d92b5768ffa18b6bc659f3f86c953,pt-br,2018-03-18T20:57:06.301Z,true,REBECCA,32,454530554
d5de3aa126ae18fd98c65fd544548cc926b36caeb8309a213631d86901bd6074,pt-br,2018-01-08T22:19:15.852Z,true,KELLY,72,47158438
fc3b69d0454f1ec8c1d196f0d4e9a7cc1409dbc87339a89c0725ec9c054a4765,pt-br,2018-01-04T02:01:17.478Z,true,ANA,91,533597575


['customer_id', 'language', 'created_at', 'active', 'customer_name', 'customer_phone_area', 'customer_phone_number']
root
 |-- customer_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- active: boolean (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_phone_area: integer (nullable = true)
 |-- customer_phone_number: integer (nullable = true)

+-------+--------------------+--------+-------------+-------------------+---------------------+
|summary|         customer_id|language|customer_name|customer_phone_area|customer_phone_number|
+-------+--------------------+--------+-------------+-------------------+---------------------+
|  count|              806156|  806156|       806156|             806156|               806156|
|   mean|                NULL|    NULL|          NaN|  49.56521194409022|  5.006510006290433E8|
| stddev|                NULL|    NULL|          NaN|  28.83715985569671| 2.884

In [0]:
#converte tipo de colunas que estão diferente do dicionario de dados
colunas_para_converter = ["customer_phone_area", "customer_phone_number"]

for c in colunas_para_converter:
    df_usuario = df_usuario.withColumn(c, col(c).cast("string"))

df_usuario.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- active: boolean (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_phone_area: string (nullable = true)
 |-- customer_phone_number: string (nullable = true)



In [0]:
#mostra nulls
display( df_usuario.select([
    count(when(col(c).isNull(), c)).alias(c + "_nulls")
    for c in df_usuario.columns
]))

customer_id_nulls,language_nulls,created_at_nulls,active_nulls,customer_name_nulls,customer_phone_area_nulls,customer_phone_number_nulls
0,0,0,0,0,0,0


**Entendimento df_restaurante**

In [0]:
#entendimento df
print((df_restaurante.count(), len(df_usuario.columns)))
display(df_restaurante.limit(10))
display(df_restaurante.columns)
df_restaurante.printSchema()
display(df_restaurante.describe())

(7292, 7)


id,created_at,enabled,price_range,average_ticket,takeout_time,delivery_time,minimum_order_value,merchant_zip_code,merchant_city,merchant_state,merchant_country
d19ff6fca6288939bff073ad0a119d25c0365c407e9e5dd999e7a3e53c6d5d76,2017-01-23T12:52:30.910Z,false,3,60.0,0,50,30.0,14025,RIBEIRAO PRETO,SP,BR
631df0985fdbbaf27b9b031a8f381924e3483833385748c8f7e1a41a6891d2b9,2017-01-20T13:14:48.286Z,true,3,60.0,0,0,30.0,50180,SAO PAULO,SP,BR
135c5c4ae4c1ec1fdc23e8c649f313e39be8db913d8bc5aaff9bfddb548f096d,2017-01-23T12:46:33.457Z,true,5,100.0,0,45,10.0,23090,RIO DE JANEIRO,RJ,BR
d26f84c470451f752bef036c55517b6d9950d41806f10e94e8b5734241468768,2017-01-20T13:15:04.806Z,true,3,80.0,0,0,18.9,40255,SALVADOR,BA,BR
97b9884600ea7192314580d9115f8882b8634f5aa201ff9a6df206e49343a283,2017-01-20T13:14:27.701Z,true,3,60.0,0,0,25.0,64600,BARUERI,SP,BR
1ec1343b97cae3e3fc561926425f5fa9ee514f4cbcc99b8b3d0689af3b1e8197,2017-01-23T12:36:59.158Z,false,1,30.0,0,60,10.0,18030,SOROCABA,SP,BR
a296322ddb834e65ac755cc1fa1cf1957d4f6134ef088419024a361640aa286a,2017-01-23T12:40:20.431Z,false,1,30.0,0,70,11.0,13208,JUNDIAI,SP,BR
283f457637aa69ef68df6d8ca077349bf86219c01f918ba1ffcbcbefcf944aac,2017-01-20T13:14:31.699Z,true,4,80.0,0,30,24.0,34020,SAO PAULO,SP,BR
d171f94340260767e67ae9a520ae9881949dba7945771d5248707bc07847f05f,2017-01-20T13:13:05.136Z,false,5,81.0,30,15,10.0,45380,SAO PAULO,SP,BR
1b8219cc06c6b99de0cfd80d2fef350ed0494ad88ceab0e643238da12dbf2559,2017-01-20T13:15:57.202Z,false,1,30.0,0,30,0.0,60541,FORTALEZA,CE,BR


_1
id
created_at
enabled
price_range
average_ticket
takeout_time
delivery_time
minimum_order_value
merchant_zip_code
merchant_city


root
 |-- id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- enabled: boolean (nullable = true)
 |-- price_range: integer (nullable = true)
 |-- average_ticket: double (nullable = true)
 |-- takeout_time: integer (nullable = true)
 |-- delivery_time: integer (nullable = true)
 |-- minimum_order_value: double (nullable = true)
 |-- merchant_zip_code: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- merchant_country: string (nullable = true)



summary,id,price_range,average_ticket,takeout_time,delivery_time,minimum_order_value,merchant_zip_code,merchant_city,merchant_state,merchant_country
count,7292,7292,7292,7292,7291,7197,7292,7292,7292,7292
mean,null,2.561300054854635,53.72682391662095,3.1839001645639056,28.034974626251543,28.79775878838398,46125.42992320351,null,null,null
stddev,null,1.1800935917985063,19.562103131436725,8.422497889627026,25.246657227480515,1178.6427269230835,25936.93864653852,null,null,null
min,000ec327878b6c6f1d3a07ac30cae115eb6cc7aa39492b5cc5ca48d0ff73b18e,1,0.0,0,0,0.0,0,AMERICANA,AC,BR
max,fff6b9b86342869089e154273cf45a3a4533c4231b31b06845d49ad777b3586a,5,100.0,60,285,100000.0,99920,VOLTA REDONDA,SP,BR


In [0]:
#converte tipo de colunas que estão diferente do dicionario de dados
colunas_para_converter = ["takeout_time", "delivery_time"]

for c in colunas_para_converter:
    df_restaurante = df_restaurante.withColumn(c, col(c).cast("double"))

df_restaurante = df_restaurante.withColumn("merchant_zip_code", col("merchant_zip_code").cast("string"))
df_restaurante.printSchema()

root
 |-- id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- enabled: boolean (nullable = true)
 |-- price_range: integer (nullable = true)
 |-- average_ticket: double (nullable = true)
 |-- takeout_time: double (nullable = true)
 |-- delivery_time: double (nullable = true)
 |-- minimum_order_value: double (nullable = true)
 |-- merchant_zip_code: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- merchant_country: string (nullable = true)



In [0]:
#mostra nulls
display( df_restaurante.select([
    count(when(col(c).isNull(), c)).alias(c + "_nulls")
    for c in df_restaurante.columns
]))

id_nulls,created_at_nulls,enabled_nulls,price_range_nulls,average_ticket_nulls,takeout_time_nulls,delivery_time_nulls,minimum_order_value_nulls,merchant_zip_code_nulls,merchant_city_nulls,merchant_state_nulls,merchant_country_nulls
0,0,0,0,0,0,1,95,0,0,0,0


**Entendimento df_teste**

In [0]:
#entendimento df
print((df_teste.count(), len(df_teste.columns)))
display(df_teste.limit(10))
display(df_teste.columns)
df_teste.printSchema()
display(df_teste.describe())

(806467, 2)


customer_id,is_target
755e1fa18f25caec5edffb188b13fd844b2af8cf5adedcf77c028f36cb9382ea,target
b821aa8372b8e5b82cdc283742757df8c45eecdd72adf411716e710525d4edf1,control
d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc007f5297541ec40b63b,control
6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c8b535718bbc651ab02,target
dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f5e3763fa657c23a58d,control
82905a7d1fe78f80ceeab4e3f523c9a3fb1a0324a1c9c6078191fd67f0f6bceb,target
2c793590086a05d39f1fdf9513f585dd7ca6d2ba404bb40248d3102e024920b2,target
2c2c1ec8e79d1b98e67aaefb39b477dd9c95bea6e498c8ff2ae610cbddd0cc17,target
5994c3c2c48edd8a7b56962761581282f03fa9479847f44bf9be27fa80803018,target
657d1330fd2b28abd8390fae089675e88fdfb76ad09b84a12c3be43e451c6acf,control


_1
customer_id
is_target


root
 |-- customer_id: string (nullable = true)
 |-- is_target: string (nullable = true)



summary,customer_id,is_target
count,806467,806467
mean,null,null
stddev,null,null
min,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e12402be6b07a099361,control
max,null,target


In [0]:
#mostra nulls
display( df_teste.select([
    count(when(col(c).isNull(), c)).alias(c + "_nulls")
    for c in df_teste.columns
]))

customer_id_nulls,is_target_nulls
0,0


**Entendimento df_pedido**

In [0]:
#entendimento df
print((df_pedido.count(), len(df_pedido.columns)))
display(df_pedido.limit(10))
display(df_pedido.columns)
df_pedido.printSchema()
display(df_pedido.describe())

(3670826, 21)


cpf,customer_id,customer_name,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,delivery_address_zip_code,items,merchant_id,merchant_latitude,merchant_longitude,merchant_timezone,order_created_at,order_id,order_scheduled,order_total_amount,origin_platform
80532101763,7ba88a68bb2a3504c6bd37a707af57a0b8d6e110a551c719c9e7f71aaf3a99cf,GUSTAVO,FRANCA,BR,JARDIM ESPRAIADO,6736655,-47.39,-20.55,SP,14403,"[{""name"": ""Parmegiana de Filé de Frango (2 pessoas)"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 1, ""unitPrice"": {""value"": ""2800"", ""currency"": ""BRL""}, ""externalId"": ""0bcd6764fd5e466d9c04b18ac0eb69e6"", ""totalValue"": {""value"": ""2800"", ""currency"": ""BRL""}, ""customerNote"": null, ""garnishItems"": [{""name"": ""COM Arroz branco"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 2, ""unitPrice"": {""value"": ""0"", ""currency"": ""BRL""}, ""categoryId"": ""13HDH"", ""externalId"": ""384bd2b4eb7d454d8e0274e7d590ab4f"", ""totalValue"": {""value"": ""0"", ""currency"": ""BRL""}, ""categoryName"": ""PERSONALIZAR"", ""integrationId"": null}], ""integrationId"": ""PMFR"", ""totalAddition"": {""value"": ""0"", ""currency"": ""BRL""}, ""totalDiscount"": {""value"": ""0"", ""currency"": ""BRL""}}, {""name"": ""Lasanha Frango (2 pessoas)"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 3, ""unitPrice"": {""value"": ""1800"", ""currency"": ""BRL""}, ""externalId"": ""a361f5eec6a44ac0817892e81bf22e80"", ""totalValue"": {""value"": ""1800"", ""currency"": ""BRL""}, ""customerNote"": null, ""garnishItems"": [], ""integrationId"": ""LF"", ""totalAddition"": {""value"": ""0"", ""currency"": ""BRL""}, ""totalDiscount"": {""value"": ""0"", ""currency"": ""BRL""}}]",a992a079a651e699d9149423761df2427c0e3af0a2a1b5bb6d2bad1cb3a3a265,-47.39,-20.55,America/Sao_Paulo,2019-01-17T22:50:06.000Z,33e0612d62e5eb42aba15b58413137e441fbe906de2febd6a6c721b0e5773b44,false,46.0,ANDROID
43352103961,078acecdcf7fa89d356bfa349f14a8219db1ee161ce28ae4d9fee550c95d6fbd,MICHELLE,SANTOS,BR,CAMPO GRANDE,8759216,-46.34,-23.96,SP,11070,"[{""name"": ""Filé Mignon à Cubana"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 1, ""unitPrice"": {""value"": ""0"", ""currency"": ""BRL""}, ""externalId"": ""e0e81b2027c241cca5bf60a4768800ec"", ""totalValue"": {""value"": ""0"", ""currency"": ""BRL""}, ""customerNote"": null, ""garnishItems"": [{""name"": ""334 - 1/2 porção "", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 2, ""unitPrice"": {""value"": ""7350"", ""currency"": ""BRL""}, ""categoryId"": ""1J3T0"", ""externalId"": ""b869c10747d1417d8968d7e3e1bd7de4"", ""totalValue"": {""value"": ""7350"", ""currency"": ""BRL""}, ""categoryName"": ""escolha sua porção "", ""integrationId"": null}], ""integrationId"": null, ""totalAddition"": {""value"": ""0"", ""currency"": ""BRL""}, ""totalDiscount"": {""value"": ""0"", ""currency"": ""BRL""}}, {""name"": ""603- Pudim de leite"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 4, ""unitPrice"": {""value"": ""800"", ""currency"": ""BRL""}, ""externalId"": ""4436f76edc164052aaf3c21a6567b570"", ""totalValue"": {""value"": ""800"", ""currency"": ""BRL""}, ""customerNote"": null, ""garnishItems"": [], ""integrationId"": null, ""totalAddition"": {""value"": ""0"", ""currency"": ""BRL""}, ""totalDiscount"": {

_1
cpf
customer_id
customer_name
delivery_address_city
delivery_address_country
delivery_address_district
delivery_address_external_id
delivery_address_latitude
delivery_address_longitude
delivery_address_state


root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: string (nullable = true)
 |-- merchant_longitude: string (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_total_amount: double (nullable = true)
 |-- o

summary,cpf,customer_id,customer_name,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,delivery_address_zip_code,items,merchant_id,merchant_latitude,merchant_longitude,merchant_timezone,order_created_at,order_id,order_total_amount,origin_platform
count,3670826,3662321,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670826,3670824
mean,5.005544976926936E10,null,NaN,null,null,null,7235483.074816131,-44.93203929033961,-20.328647726696943,null,42831.187055447466,null,null,-44.93203929033961,-20.328647726696943,null,null,null,47.90005515165806,null
stddev,2.8837294468561066E10,null,NaN,null,null,null,2111851.456950484,4.438217743782714,6.53610336563565,null,24495.48478150103,null,null,4.438217743782714,6.53610336563565,null,null,null,119.67384637744763,null
min,00013286992,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e12402be6b07a099361,,ALMIRANTE TAMANDARE,BR,,1051625,-34.79,-1.07,AC,0,"[{""name"": "" MÉDIA (06 PEDAÇOS)"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 1, ""unitPrice"": {""value"": ""0"", ""currency"": ""BRL""}, ""externalId"": ""3d912149ff2d43579ee3fa5d47a076f9"", ""totalValue"": {""value"": ""0"", ""currency"": ""BRL""}, ""customerNote"": null, ""garnishItems"": [{""name"": ""MASSA MÉDIA - TRADICIONAL"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 2, ""unitPrice"": {""value"": ""0"", ""currency"": ""BRL""}, ""categoryId"": ""0025"", ""externalId"": ""b3a695a6b9bd4249a2580b8ed504cfb0"", ""totalValue"": {""value"": ""0"", ""currency"": ""BRL""}, ""categoryName"": ""Escolha a sua Preferência"", ""integrationId"": null}, {""name"": ""BRIGADEIRO"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 1.00, ""sequence"": 3, ""unitPrice"": {""value"": ""3300"", ""currency"": ""BRL""}, ""categoryId"": ""SBR"", ""externalId"": ""8eea5f7ea5274361843439fafd10cdd1"", ""totalValue"": {""value"": ""3300"", ""currency"": ""BRL""}, ""categoryName"": ""Escolha um sabor"", ""integrationId"": null}], ""integrationId"": null, ""totalAddition"": {""value"": ""0"", ""currency"": ""BRL""}, ""totalDiscount"": {""value"": ""0"", ""currency"": ""BRL""}}]",000ec327878b6c6f1d3a07ac30cae115eb6cc7aa39492b5cc5ca48d0ff73b18e,-34.79,-1.07,America/Bahia,2018-12-03T00:00:00.000Z,000004f018699e8aa88513e8e3e5394c0f33a4720d78f273b49e2b4c4b45c0ca,0.0,ANDROID
max,99999989150,ffffeda85b466b48195c356f8b97f21d0b23958d5815fcd401d62f0f4c426195,•fanilo,VOTORANTIM,BR,ZUMBI DOS PALMARES,9547214,0.00,38.71,SP,99933,"[{""name"": ""• Refrigerante"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 2.00, ""sequence"": 5, ""unitPrice"": {""value"": ""450"", ""currency"": ""BRL""}, ""externalId"": ""99d2de7fb0e9407d8de4f23c5e635723"", ""totalValue"": {""value"": ""900"", ""currency"": ""BRL""}, ""customerNote"": null, ""garnishItems"": [{""name"": ""Coca cola zero"", ""addition"": {""value"": ""0"", ""currency"": ""BRL""}, ""discount"": {""value"": ""0"", ""currency"": ""BRL""}, ""quantity"": 2.00, ""sequence"": 6, ""unitPrice"": {""value"": ""0"", ""currency"": ""BRL""}, ""categoryId"": ""1K2YI"", ""externalId"": ""2e5e20366bea4df5a1b1bee0a05354e5"", ""totalValue"": {""value"": ""0"", ""currency"": ""BRL""}, ""categoryName"": ""Escolha o refrigerante de sua preferência"", ""integrationId"": null}], ""integrationId"": null, ""totalAddition"": {""value"": ""0"", ""currency"": ""BRL""}, ""totalDiscount"": {""value"": ""0"", ""currency"": ""BRL""}}, {""name"": ""GRANDE 2 SABORES"", ""addition"": {""value"": ""

In [0]:
#converte tipo de colunas que estão diferente do dicionario de dados
colunas_para_converter = ["delivery_address_latitude", "delivery_address_latitude", "merchant_latitude", "merchant_longitude"]

for c in colunas_para_converter:
    df_pedido = df_pedido.withColumn(c, col(c).cast("double"))

df_pedido = df_pedido.withColumn("order_created_at", col("order_created_at").cast("timestamp"))
df_pedido.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: double (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: double (nullable = true)
 |-- merchant_longitude: double (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: timestamp (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_total_amount: double (nullable = true)
 |-

Ajuste df_estado

In [0]:
#alteracao DDD estados
df_estado = df_estado.withColumn("DDD", col("DDD").cast("string"))

In [0]:
#salvar em delta
df_pedido.write.format("delta").mode("overwrite").save(base_path + "process/pedido")
df_restaurante.write.format("delta").mode("overwrite").save(base_path + "process/restaurante")
df_usuario.write.format("delta").mode("overwrite").save(base_path + "process/usuario")
df_teste.write.format("delta").mode("overwrite").save(base_path + "process/teste")
df_estado.write.format("delta").mode("overwrite").save(base_path + "process/estado")